In [ ]:
#1 Loading the Data
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
if os.path.isdir('Graphs') == False:
    os.mkdir('Graphs')
burglar_df = pd.read_csv('Burglars.csv')

In [ ]:
#2 Grouping by LSOA
groups = burglar_df.groupby('LSOA name')
grouped_df = {name: df for name, df in groups}

In [ ]:
#3 Define month-grabbing function:
def get_per_month_count(unf_df, fil_df):
    months = unf_df['Month'].unique()
    per_month = {}
    for i in months:
        per_month[i] = len(fil_df[fil_df['Month'] == i])
    return per_month

In [ ]:
#4 Generating the per-month-year bar charts and dictionaries:
LSOA_per_month = {}
for i in grouped_df.keys():
    df = grouped_df[i]
    groups = df.groupby('Month')
    month_df = get_per_month_count(burglar_df, df)
    LSOA_per_month[i] = month_df
    months = list(month_df.keys())
    step = max(1, len(months) // 15)


In [ ]:
#5 Automatic Recency problem analysis: Define Capturing of Points and Gaps
print(LSOA_per_month.keys())
point_gap = {}
for i in LSOA_per_month.keys():
    df = LSOA_per_month[i]
    gap = []
    point = []
    neither = []
    for month in df.keys():
        if df[month] == 0:
            gap.append(month)
        elif df[month] == 1:
            point.append(month)
        else:
            neither.append(month)
    point_gap[i] = (point, gap, neither)
    print(f'LSOA {i}')


In [ ]:
durations = {}
df = burglar_df['Month'].unique()
len_ = len(point_gap.keys())
count = 0
prevp = 0
for i in point_gap.keys():
    print(i)
    distance = []
    point = point_gap[i][0]
    gap = point_gap[i][1]
    neither = point_gap[i][2]
    for month in df:
        if month in point:
            to_gap = 0
            dt_month = datetime.strptime(month, '%Y-%m')
            gap_c = [datetime.strptime(date, '%Y-%m') for date in gap]
            for j in range(12):
                current_date = dt_month + timedelta(days=30*j)  # adding approximately a month each time

                # Check if the current date is in the gap list
                if current_date in gap_c:
                    current_string = current_date.strftime('%Y-%m')
                    print(f'Point at {month}, Gap at {current_string}.')
                    distance.append((dt_month, to_gap))
                    break
                else:
                    to_gap += 1
    durations[i] = distance
    


In [ ]:
#6 Calculate mean and stdev distance
mean_durations = {}
for i in durations.keys():
    mean_durations[i] = [(np.mean(durations[i]), np.std(durations[i])) if len(durations[i]) != 0 else 0][0]
print(mean_durations)